# Disclaimer 
This section should not be run as it takes too much time and results in a 30Gb file with all the 3d volumes concatenated after having been coregistered. Due to limited storage space on the VM we decided not to include this in our preprocessing pipeline but given the effort put into it, to leave it here for the corrector to appreciate the effort nonetheless.


In [ ]:
%gui wx
%load_ext autoreload
%autoreload 2
import sys
import os
import os.path as op
import glob

# Import of utils.py functions
current_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(current_dir); sys.path.append(parent_dir); sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist

# Loading fsl and freesurfer within Neurodesk
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

# FSL function wrappers which we will call from python directly
loadFSL() # Setup FSL path
from fsl.wrappers import fast, bet, flirt, fslmaths
from fsl.wrappers.misc import fslroi

import mne
import nibabel as nib
import nilearn
import subprocess
import ants
import openneuro

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from preprocessed import viz_fsleyes

## Tissue segmentation using fast

In [ ]:
from preprocessed import apply_fast

segmentation_path = apply_fast(preproc_path, subject)
pve0 = glob.glob(op.join(preproc_path, subject, 'anat','*pve_0*'))[0]
pve1 = glob.glob(op.join(preproc_path, subject, 'anat','*pve_1*'))[0]
pve2 = glob.glob(op.join(preproc_path, subject, 'anat','*pve_2*'))[0]
viz_fsleyes(fsleyesDisplay, [betted_brain_path, pve0, pve1, pve2], viz=viz)

## Epi to anatomical coregistration (not implemented)

In [ ]:
from preprocessed import apply_epi_reg
epi_reg_path, reference_epi = apply_epi_reg(dataset_path, preproc_path, path_moco_data, subject, task, 'all')

# We did the coregistration of the reference volume and get a transform, now we need to apply it to all volumes

In [ ]:
# Inspect if the transformation worked

viz_fsleyes(fsleyesDisplay, [betted_brain_path, epi_reg_path], viz=viz)

In [ ]:
#Check how the white matter of the EPI matches the anatomical's white matter

viz_fsleyes(fsleyesDisplay, 
            ['./data/derivatives/preprocessed_data/sub-control01/func/sub-control01_task-music_run-all_bold_anat-space_epi_fast_wmseg.nii.gz', './data/derivatives/preprocessed_data/sub-control01/anat/sub-control01_T1w_fast_pve_2'], 
            viz=viz)


In [ ]:
splits_epi_path = op.join(preproc_path, subject, 'func','splits_epi')
mkdir_no_exist(splits_epi_path)

start = 0

for i, split_vol in tqdm(enumerate(split_vols[start:]), initial = start, total = 315): 
    split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
    out_vol = op.join(splits_epi_path,'sub-control01_task-music_run-all_bold_epi_vol' + split_nbr) # or output to desktop
    
    subprocess.run(['flirt', '-in', split_vol, '-ref', anatomical_path,'-out', out_vol,
                        '-init', path_epi_transform,'-applyxfm'])
os.system('rm -rf {}'.format(op.join(splits_path, '*_bold_split*'))) # remove the splits from disk

In [ ]:
from preprocessed import combine_splits

epi_all = combine_splits(preproc_path, splits_epi_path, subject)
#Compare against anatomical path
viz_fsleyes(fsleyesDisplay, [anatomical_path, epi_all], viz=viz)
